In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
goodgov= pd.read_csv("../input/What Makes a Good Government.csv")
goodgov=goodgov.drop(['Unnamed: 10','Unnamed: 22','Unnamed: 4','indicator','ISO Country code'],axis=1).loc[4:198]
goodgov.head(10)

In [ ]:
goodgov.columns = goodgov.columns.str.replace(" ","_").str.replace("(","").str.replace(")","").str.replace("\r\n","").str.replace("%","")
goodgov.head()

In [ ]:
cols = goodgov.columns[goodgov.dtypes.eq(object)]
goodgov[cols] = goodgov[cols].apply(pd.to_numeric, errors='coerce', axis=0)
print(list(cols))

In [ ]:
x1 = goodgov.dropna(subset=['happy_planet_index'])
y1 = x1.happy_planet_index
x1 = x1.drop(['world_happiness_report_score','human_development_index','happy_planet_index'],axis=1)
x1 = x1.fillna(x1.mean())


x2 = goodgov.dropna(subset=['human_development_index'])
y2 = x2.human_development_index
x2 = x2.drop(['world_happiness_report_score','human_development_index','happy_planet_index'],axis=1)
x2 = x2.fillna(x2.mean())

x3 = goodgov.dropna(subset=['world_happiness_report_score'])
y3 = x3.world_happiness_report_score
x3 = x3.drop(['world_happiness_report_score','human_development_index','happy_planet_index'],axis=1)
x3 = x3.fillna(x3.mean())

In [ ]:
import lightgbm as lgbm
import xgboost as xgb
## Import the random forest model.
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test = train_test_split(x1,y1,random_state=123,test_size=0.05)


## This line instantiates the model. 
rf = RandomForestRegressor() 
## Fit the model on your training data.
rf.fit(x_train, y_train) 

features = list(x1.columns)
importances = rf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.rcParams["figure.figsize"] = [30,25]
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 25}

plt.rc('font', **font)

plt.show()

In [ ]:
import lightgbm as lgbm
import xgboost as xgb
## Import the random forest model.
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test = train_test_split(x2,y2,random_state=123,test_size=0.05)


## This line instantiates the model. 
rf = RandomForestRegressor() 
## Fit the model on your training data.
rf.fit(x_train, y_train) 

features = list(x1.columns)
importances = rf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.rcParams["figure.figsize"] = [30,25]
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 25}

plt.rc('font', **font)
plt.show()

In [ ]:
import lightgbm as lgbm
import xgboost as xgb
## Import the random forest model.
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test = train_test_split(x3,y3,random_state=123,test_size=0.05)


## This line instantiates the model. 
rf = RandomForestRegressor() 
## Fit the model on your training data.
rf.fit(x_train, y_train) 

features = list(x1.columns)
importances = rf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.rcParams["figure.figsize"] = [30,25]
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 25}

plt.rc('font', **font)
plt.show()

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances 

In [ ]:
#xgb.plot_importance(model,max_num_features=4)

In [ ]:
import xgboost as xgb  

x_train,x_test,y_train,y_test
dtrain = xgb.DMatrix(x_train, label=y_train,missing=999.0)
dvalid = xgb.DMatrix(x_test, label=y_test,missing=999.0)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Try different parameters! My favorite is random search :)
xgb_pars = {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.8, 'max_depth': 15,
            'subsample': 0.75, 'lambda': 2, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}  

model2 = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=50,
                  maximize=False, verbose_eval=15)

In [ ]:


#feature_names=['population','surface area_Km2','GINI_index']


dtrain = xgb.DMatrix(x_train, label=y_train#,missing=-999.0,feature_names=feature_names)
                    )
dvalid = xgb.DMatrix(x_test, label=y_test,#missing=-999.0,feature_names=feature_names)
                    )
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Try different parameters! My favorite is random search :)
xgb_pars = {'min_child_weight': 10, 'eta': 0.04, 'colsample_bytree': 0.8, 'max_depth': 15,
           'subsample': 0.75, 'lambda': 2, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
           'eval_metric': 'rmse', 'objective': 'reg:linear'}  

model = xgb.train(xgb_pars, dtrain, 500, watchlist, early_stopping_rounds=14,
                maximize=False, verbose_eval=15)
#model.save_model('0001.model')

In [ ]:

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 4,
        'num_leaves': 31,
        'learning_rate': 0.01,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 5000 ,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0,
        'num_rounds':1000
    }
    
train_set = lgbm.Dataset(x_train, y_train#, silent=False,categorical_feature=['year','month','day','weekday'])
                        )
valid_set = lgbm.Dataset(x_test, y_test#, silent=False#,categorical_feature=['year','month','day','weekday'])
                        )
model = lgbm.train(params, train_set = train_set, num_boost_round=1000,early_stopping_rounds=50,verbose_eval=500, valid_sets=valid_set)